<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 ../../data/stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l ../../data/stackoverflow.10kk.tsv

 10000000 ../../data/stackoverflow.10kk.tsv
CPU times: user 526 ms, sys: 183 ms, total: 709 ms
Wall time: 26.5 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [4]:
!python preprocess.py ../../data/stackoverflow.10kk.tsv ../../data/stackoverflow.vw

10000000it [02:23, 69815.29it/s]


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [5]:
!head -1 ../../data/stackoverflow.vw
!wc -l ../../data/stackoverflow.vw

!split -l 1463018 ../../data/stackoverflow.vw
!mv xaa ../../data/stackoverflow_train.vw
!mv xab ../../data/stackoverflow_test.vw
!mv xac ../../data/stackoverflow_valid.vw
!rm ../../data/stackoverflow.vw

!rm ../../data/stackoverflow_test_labels.txt 2> /dev/null
!rm ../../data/stackoverflow_valid_labels.txt 2> /dev/null

!cat ../../data/stackoverflow_test.vw | tqdm | cut -d' ' -f1 > ../../data/stackoverflow_test_labels.txt
!cat ../../data/stackoverflow_valid.vw | tqdm | cut -d' ' -f1 > ../../data/stackoverflow_valid_labels.txt

1 |text i ve got some code in window scroll that checks if an element is visible then triggers another function however only the first section of code is firing both bits of code work in and of themselves if i swap their order whichever is on top fires correctly my code is as follows fn isonscreen function use strict var win window viewport top win scrolltop left win scrollleft bounds this offset viewport right viewport left + win width viewport bottom viewport top + win height bounds right bounds left + this outerwidth bounds bottom bounds top + this outerheight return viewport right lt bounds left viewport left gt bounds right viewport bottom lt bounds top viewport top gt bounds bottom window scroll function use strict var load_more_results ajax load_more_results isonscreen if load_more_results true loadmoreresults var load_more_staff ajax load_more_staff isonscreen if load_more_staff true loadmorestaff what am i doing wrong can you only fire one event from window scroll i assume not

### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [6]:
def accuracy_on(true_file, predict_file):
    y_true = np.loadtxt(true_file)
    y_pred = np.loadtxt(predict_file)
    return accuracy_score(y_true, y_pred)


def accuracy_on_validation(predict_file):
    return accuracy_on('../../data/stackoverflow_valid_labels.txt', predict_file)


def accuracy_on_test(predict_file):
    return accuracy_on('../../data/stackoverflow_test_labels.txt', predict_file)

In [7]:
%%time
!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 1 --ngram 1 -b 28 --random_seed 17 \
    -f ../../data/so_model_1u.vw --quiet
!vw -t -i ../../data/so_model_1u.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("1u:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 1 --ngram 2 -b 28 --random_seed 17 \
    -f ../../data/so_model_1b.vw --quiet
!vw -t -i ../../data/so_model_1b.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("1b:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 1 --ngram 3 -b 28 --random_seed 17 \
    -f ../../data/so_model_1t.vw --quiet
!vw -t -i ../../data/so_model_1t.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("1t:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 3 --ngram 1 -b 28 --random_seed 17 \
    -f ../../data/so_model_3u.vw --quiet
!vw -t -i ../../data/so_model_3u.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("3u:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 3 --ngram 2 -b 28 --random_seed 17 \
    -f ../../data/so_model_3b.vw --quiet
!vw -t -i ../../data/so_model_3b.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("3b:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 3 --ngram 3 -b 28 --random_seed 17 \
    -f ../../data/so_model_3t.vw --quiet
!vw -t -i ../../data/so_model_3t.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("3t:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 5 --ngram 1 -b 28 --random_seed 17 \
    -f ../../data/so_model_5u.vw --quiet
!vw -t -i ../../data/so_model_5u.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("5u:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 5 --ngram 2 -b 28 --random_seed 17 \
    -f ../../data/so_model_5b.vw --quiet
!vw -t -i ../../data/so_model_5b.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("5b:", accuracy_on_validation('../../data/predict.csv'))

!vw --oaa 10 -c -d ../../data/stackoverflow_train.vw --passes 5 --ngram 3 -b 28 --random_seed 17 \
    -f ../../data/so_model_5t.vw --quiet
!vw -t -i ../../data/so_model_5t.vw -d ../../data/stackoverflow_valid.vw -p ../../data/predict.csv --quiet
print("5t:", accuracy_on_validation('../../data/predict.csv'))

!rm ../../data/stackoverflow_train.vw.cache 2> /dev/null

1u: 0.915151419873
1b: 0.930828602245
1t: 0.929616723786
3u: 0.914390663683
3b: 0.928217561233
3t: 0.926844372386
5u: 0.913746789171
5b: 0.92932417783
5t: 0.927335138734
CPU times: user 2min 52s, sys: 14.5 s, total: 3min 7s
Wall time: 47min 8s


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- **Биграммы и 1 проход по выборке**
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [8]:
!vw -t -i ../../data/so_model_1b.vw -d ../../data/stackoverflow_test.vw -p ../../data/predict.csv --quiet
train_acc = accuracy_on_test('../../data/predict.csv')
print("[Train] Accuracy on test:", train_acc)

[Train] Accuracy on test: 0.930669342414


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- **Результаты почти одинаковы – отличаются меньше чем на 0.5%**

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [9]:
!rm ../../data/stackoverflow_train_valid.vw 2> /dev/null
!cat ../../data/stackoverflow_train.vw ../../data/stackoverflow_valid.vw > ../../data/stackoverflow_train_valid.vw

!vw --oaa 10 -d ../../data/stackoverflow_train_valid.vw --passes 1 --ngram 2 -b 28 --random_seed 17 \
    -f ../../data/so_model_2x_1b.vw
!vw -t -i ../../data/so_model_2x_1b.vw -d ../../data/stackoverflow_test.vw  -p ../../data/predict.csv --quiet

train_val_acc = accuracy_on_test('../../data/predict.csv')
print("\n[Train + Val] Accuracy on test: {:.5}".format(train_val_acc))
print("Diff: {:.2}%".format((train_val_acc - train_acc) * 100))

Generating 2-grams for all namespaces.
final_regressor = ../../data/so_model_2x_1b.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/stackoverflow_train_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.765625 0.750000           64           64.0        3        3      406
0.671875 0.578125          128          128.0        1        7       56
0.597656 0.523438        

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - **0.4%**
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)